In [9]:
#intraday check at the end of the day

import yfinance as yf
import pandas as pd
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

#fetching stock list from excel
excel_data_df = pd.read_excel('NSE\Monitored_Tickers.xlsx', sheet_name='Sheet1')
indian_tickers = excel_data_df['Ticker'].tolist()

# Function to check if the stock meets the criteria
def check_stock(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="max", interval="1d")
    #hist = stock.history(period="1y", interval="1d")
    
    if hist.empty or len(hist) < 2:
        return None
    
    current_price = hist['Close'][-1]
    all_time_high_till_yesterday = hist['High'][:-1].max()  # All-time high till yesterday
    todays_high = hist['High'][-1]
    todays_open = hist['Open'][-1]
    #breakout = FTweekhigh if all_time_high_till_yesterday > FTweekhigh else all_time_high_till_yesterday    
    
    if 50 < current_price < 3000:
        if current_price >= all_time_high_till_yesterday * 0.80:
        #if todays_high >= all_time_high_till_yesterday:# * 0.98:
            #volume = hist['Volume'][-1]
            #cmp_ath = current_price - all_time_high_till_yesterday
            #todays_high_cmp = todays_high - current_price

            return {
                'Ticker': ticker,
            }
    
    return None

# List to store the stocks that meet the criteria
filtered_stocks = []

# Loop through the tickers and apply the filter
for ticker in indian_tickers:
    result = check_stock(ticker)
    if result:
        filtered_stocks.append(result)

# Create a DataFrame from the results
filtered_stocks_df = pd.DataFrame(filtered_stocks)

# Sort the DataFrame by Volume in descending order and get the top 25 stocks
#filtered_stocks_df = filtered_stocks_df.sort_values(by='Volume', ascending=False)#.head(25)

# Calculate the sum of the 'CMP-ATH' column
#sum_cmp_ath = filtered_stocks_df['Profit'].sum()

# Print the DataFrame and the sum of 'CMP-ATH'
if not filtered_stocks_df.empty:
    print(filtered_stocks_df)
    #print(f"\nSum of 'CMP-ATH' column: {sum_cmp_ath}")
    filtered_stocks_df.to_csv('Short_List_Twenty.csv', index=False)
else:
    print("No stocks meet the criteria.")

$ABNINT.NS: possibly delisted; No timezone found
$AKANKSHA.NS: possibly delisted; No timezone found
$ARHAM.NS: possibly delisted; No timezone found
$BDR.NS: possibly delisted; No price data found  (1d 1925-08-22 -> 2024-07-29)


$BDR.NS: possibly delisted; No price data found  (1d 1925-08-22 -> 2024-07-29)


$BHALCHANDR.NS: possibly delisted; No timezone found
$EASTSUGIND.NS: possibly delisted; No timezone found
$FOCE.NS: possibly delisted; No timezone found
$GAMMONIND.NS: possibly delisted; No timezone found
$HINDZINC.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HIRECT.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HISARMETAL.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HITECH.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HITECHCORP.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HITECHGEAR.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HLEGLAS.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HLVLTD.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HMAAGRO.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HMT.NS: possibly delisted; No price data

            Ticker
0     20MICRONS.NS
1    21STCENMGM.NS
2        360ONE.NS
3         AARON.NS
4       AARTECH.NS
..             ...
761      ZODIAC.NS
762      ZOMATO.NS
763        ZOTA.NS
764   ZYDUSLIFE.NS
765   ZYDUSWELL.NS

[766 rows x 1 columns]


In [10]:
import yfinance as yf
import pandas as pd
import warnings
import datetime
from tqdm import tqdm

# Ignore warnings
warnings.filterwarnings('ignore')

# Load stock list from Excel file
excel_file_path = 'Short_List_Twenty.xlsx'
sheet_name = 'Short_List_Twenty'

# Read the Excel file into a DataFrame
stock_list_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Extract the 'Ticker' column and convert it to a list
indian_stock_list = stock_list_df['Ticker'].tolist()

# Function to check stock details and identify profitable stocks
def check_stock(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="max", interval="1d")
    keys_to_extract = [
        'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'maxAge', 'priceHint',
        'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 
        'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'payoutRatio', 'beta', 'trailingPE', 
        'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 
        'bid', 'ask', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 
        'fiftyDayAverage', 'twoHundredDayAverage', 'enterpriseValue', 'profitMargins', 'floatShares', 
        'sharesOutstanding', 'heldPercentInsiders', 'heldPercentInstitutions', 'impliedSharesOutstanding', 
        'bookValue', 'priceToBook', 'lastFiscalYearEnd', 'nextFiscalYearEnd', 'mostRecentQuarter', 
        'earningsQuarterlyGrowth', 'netIncomeToCommon', 'trailingEps', 'forwardEps', 'lastSplitFactor', 
        'lastSplitDate', 'enterpriseToRevenue', 'enterpriseToEbitda', '52WeekChange', 'SandP52WeekChange', 
        'lastDividendValue', 'lastDividendDate', 'shortName', 'longName', 'firstTradeDateEpochUtc', 
        'gmtOffSetMilliseconds', 'currentPrice', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 
        'targetMedianPrice', 'recommendationMean', 'recommendationKey', 'numberOfAnalystOpinions', 'totalCash', 
        'totalCashPerShare', 'ebitda', 'totalDebt', 'totalRevenue', 'debtToEquity', 'revenuePerShare', 
        'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins'
    ]
    All_data = {}
    try:
        fifty_two_week_high = stock.info['fiftyTwoWeekHigh']
    except KeyError:
        fifty_two_week_high = 0

    if hist.empty or len(hist) < 2:
        return None

    current_price = hist['Close'][-1]
    all_time_high_till_yesterday = hist['High'][:-1].max()
    todays_high = hist['High'][-1]
    todays_open = hist['Open'][-1]

    breakout_level = fifty_two_week_high if all_time_high_till_yesterday > fifty_two_week_high and fifty_two_week_high != 0 else all_time_high_till_yesterday

    if 50 < current_price < 3000 and todays_high >= breakout_level + 1:
        volume = hist['Volume'][-1]

        if todays_open > breakout_level:
            entry_price = todays_open
            stop_loss = breakout_level - 1
        else:
            entry_price = breakout_level
            stop_loss = max(todays_open, breakout_level * 0.99) - 1

        if entry_price > 100:
            Trailing_Stop_Loss = todays_high - 5
        else:
            Trailing_Stop_Loss = entry_price - (todays_high * 0.05)

        Trailing_Stop_Loss= max(Trailing_Stop_Loss,stop_loss)
        exit_price = max(current_price, stop_loss)
        stop_loss_hit = "Yes" if current_price < stop_loss else "No"
        profit = exit_price - entry_price
        extracted_values = {key: stock.info.get(key) for key in keys_to_extract}
        All_data.update(extracted_values)

        data = {
            'Ticker': ticker,
            'All Time High': all_time_high_till_yesterday,
            '52 Week High': fifty_two_week_high,
            'Breakout Level': breakout_level,
            'Today\'s High': todays_high,
            'Today\'s Open': todays_open,
            'Current Market Price': current_price,
            'Stop Loss': stop_loss,
            'Stop Loss Points': entry_price - stop_loss,
            'Stop Loss Hit': stop_loss_hit,
            'Entry Price': entry_price,
            'Exit Price': exit_price,
            'Profit': profit,
            'Trailing Exit': Trailing_Stop_Loss,
            'Trailing Profit': Trailing_Stop_Loss - entry_price
            
        }
        All_data.update(data)

    return All_data

# List to store filtered stocks
filtered_stocks = []

# Loop through the tickers and apply the filter with a progress bar
for ticker in tqdm(indian_stock_list, desc="Processing stocks"):
    result = check_stock(ticker)
    if result:
        filtered_stocks.append(result)

# Create a DataFrame from the results
filtered_stocks_df = pd.DataFrame(filtered_stocks)

# Sort the DataFrame by Volume in descending order
filtered_stocks_df = filtered_stocks_df.sort_values(by='Profit', ascending=False)

# Calculate the total profit and other metrics
total_profit = filtered_stocks_df['Profit'].sum()
total_trailing_sl = filtered_stocks_df['Trailing Exit'].sum()
total_entry_capital = filtered_stocks_df['Entry Price'].sum()
total_return_percentage = (total_profit / total_entry_capital) * 100
leverage_capital = total_entry_capital * 0.2
total_return_with_leverage = (total_profit / leverage_capital) * 100

# Print the DataFrame and calculated metrics
if not filtered_stocks_df.empty:
    print(filtered_stocks_df)
    print(f"\nToday's Profit: {total_profit}")
    print(f"\nTrailing Stop Loss High Profit: {total_trailing_sl}")
    print(f"\nTotal Capital Used: {total_entry_capital}")
    print(f"\nTotal Return for the Day: {total_return_percentage:.2f}%")
    print(f"\nWith 80% Leverage, Capital Used: {leverage_capital}")
    print(f"\nTotal Return for the Day with Leverage: {total_return_with_leverage:.2f}%")

    timestamp = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M")
    filtered_stocks_df.to_csv(f'strategy_results_{timestamp}.csv', index=False)
else:
    print("No stocks meet the criteria.")

Processing stocks: 100%|██████████| 836/836 [07:40<00:00,  1.82it/s]

    auditRisk  boardRisk  compensationRisk  shareHolderRightsRisk  \
53        NaN        NaN               NaN                    NaN   
34        NaN        NaN               NaN                    NaN   
60        NaN        NaN               NaN                    NaN   
18        NaN        NaN               NaN                    NaN   
40        NaN        NaN               NaN                    NaN   
..        ...        ...               ...                    ...   
16        NaN        NaN               NaN                    NaN   
50        NaN        NaN               NaN                    NaN   
27        NaN        NaN               NaN                    NaN   
44        9.0        2.0               1.0                    5.0   
83        NaN        NaN               NaN                    NaN   

    overallRisk  maxAge  priceHint  previousClose     open   dayLow  ...  \
53          NaN   86400        NaN            NaN      NaN      NaN  ...   
34          NaN   8